In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image
import pickle

## Data Processing

### Loading Data

In [3]:
file_path1 = '/Users/yaokaiwin/Downloads/Data/c1.pkl'
file_path2 = '/Users/yaokaiwin/Downloads/Data/c2.pkl'
file_path3 = '/Users/yaokaiwin/Downloads/Data/c3.pkl'

In [11]:
with open(file_path1, 'rb') as f:
    channel_1_with_labels = pickle.load(f)
channel_1 = channel_1_with_labels.drop(columns=['label'])
channel_1 = channel_1.to_numpy().reshape(2756, 650, 650)

In [15]:
print(channel_1.shape)

(2756, 650, 650)


In [16]:
with open(file_path2, 'rb') as f:
    channel_2_with_labels = pickle.load(f)

channel_2 = channel_2_with_labels.drop(columns=['label'])
channel_2 = channel_2.to_numpy().reshape(2756, 650, 650)

In [17]:
print(channel_2.shape)

(2756, 650, 650)


In [18]:
with open(file_path3, 'rb') as f:
    channel_3_with_labels = pickle.load(f)

channel_3 = channel_3_with_labels.drop(columns=['label'])
channel_3 = channel_3.to_numpy().reshape(2756, 650, 650)

In [19]:
print(channel_3.shape)

(2756, 650, 650)


In [23]:
label = channel_1_with_labels['label']

### Preparing Training and testing data

In [24]:
num_samples = channel_1.shape[0]
height, width = 650, 650
X = np.stack((channel_1, channel_2, channel_3), axis=-1)
y = label.to_numpy()

In [25]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (2756, 650, 650, 3)
Shape of y: (2756,)


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## CNN Model

### Model Design

In [28]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [29]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(650, 650, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dense(3, activation='softmax'))

/Applications/anaconda3/envs/cse151/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 648, 648, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 324, 324, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 322, 322, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 161, 161, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 159, 159, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 79, 79, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 798848)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │   102,252,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,354,371 (390.45 MB)

 Trainable params: 102,354,371 (390.45 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Model Training

In [32]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


2024-07-18 09:12:30.601767: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


69/69 ━━━━━━━━━━━━━━━━━━━━ 251s 4s/step - accuracy: 0.3998 - loss: 2.3966 - val_accuracy: 0.4076 - val_loss: 1.0796
Epoch 2/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 246s 4s/step - accuracy: 0.4534 - loss: 1.0599 - val_accuracy: 0.4475 - val_loss: 1.0606
Epoch 3/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 243s 4s/step - accuracy: 0.5412 - loss: 0.9767 - val_accuracy: 0.4366 - val_loss: 1.2768
Epoch 4/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 246s 4s/step - accuracy: 0.6364 - loss: 0.8057 - val_accuracy: 0.5688 - val_loss: 0.9834
Epoch 5/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 238s 3s/step - accuracy: 0.7753 - loss: 0.5861 - val_accuracy: 0.5761 - val_loss: 1.1020
Epoch 6/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 238s 3s/step - accuracy: 0.8569 - loss: 0.3839 - val_accuracy: 0.6087 - val_loss: 1.4852
Epoch 7/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 238s 3s/step - accuracy: 0.9196 - loss: 0.2552 - val_accuracy: 0.5942 - val_loss: 2.2524
Epoch 8/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 237s 3s/step - accuracy: 0.9353 - loss: 0.2614 - val_accuracy: 0.6268 - val_loss: 1.

### Model testing

In [33]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 832ms/step - accuracy: 0.5926 - loss: 3.0585
Test Loss: 3.083507537841797
Test Accuracy: 0.6086956262588501


In [34]:
from sklearn.metrics import classification_report

# Predict the labels for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print the classification report
print(classification_report(y_test, y_pred_classes, target_names=['Class 0', 'Class 1', 'Class 2']))

18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 836ms/step
              precision    recall  f1-score   support

     Class 0       0.57      0.32      0.41       131
     Class 1       0.63      0.63      0.63       221
     Class 2       0.60      0.77      0.67       200

    accuracy                           0.61       552
   macro avg       0.60      0.57      0.57       552
weighted avg       0.61      0.61      0.60       552



In [37]:
model.save('cnn_model_v1.keras')